# 元组的诡异赋值
## 一个谜题
如果执行下列程序，结果是什么？
```python
atuple = (1, 2, [1, 2])
atuple[2] += [3, 4]
```
- A) atuple会变成 ```(1, 2, [1, 2, 3, 4])```
- B) 因为元组不支持元素赋值，会报错```TypeError```
- C) AB都不是
- D) AB都是

In [7]:
atuple = (1, 2, [1, 2])
atuple[2] += [3, 4]


TypeError: 'tuple' object does not support item assignment

In [8]:
print(atuple)


(1, 2, [1, 2, 3, 4])


## 元组不支持元素赋值
- ```atuple[0] = 1``` 等价于 ```atuple.__setitem__(0, 1)```
- 元组不支持元素赋值的机制：未定义特殊方法```__setitem__(self, key, value)```

In [9]:
atuple[0] = 1

TypeError: 'tuple' object does not support item assignment

## “+=”的机制
- ```a += b``` 等价于 ```a = a.__iadd__(b)```
- 但如果a的类型未定义```___iadd__(self, other)```
- ```a += b``` 就回落为 ```a = a + b```，也就是```a = a.__add__(b)```
- 如果a的类型连```__add__(self, other)```都没定义，也不要紧
- 再试试```b.__radd__(a)```
- 如果b的类型没定义```__radd__(self, other)```，才导致```a += b```计算失败

In [10]:
# 列表
alist = [1, 2]
alist += [3, 4]
print(alist)
alist = alist.__iadd__([5, 6])
print(alist)


[1, 2, 3, 4]
[1, 2, 3, 4, 5, 6]


In [11]:
# 整数
a = 12
a += 10
print(a)
a = a.__iadd__(10)
print(a)


22


AttributeError: 'int' object has no attribute '__iadd__'

## 解谜时刻
- ```atuple[2] += [3, 4]``` 等价于下面三条语句

In [12]:
result = atuple.__getitem__(2)
result = result.__iadd__([3, 4])
atuple.__setitem__(2, result)

AttributeError: 'tuple' object has no attribute '__setitem__'

In [13]:
result

[1, 2, 3, 4, 3, 4]

## 改变元组中的列表内容的正确姿势
- 不要给元组元素赋值，直接调用列表方法；

In [14]:
# 直接调用列表方法
atuple = (1, 2, [1, 2])
print(atuple)
atuple[2].pop()
print(atuple)
atuple[2].append(99)
print(atuple)
atuple[2].extend([0, 2])  # += 调用了 extend 方法
print(atuple)
atuple[2].sort()
print(atuple)


(1, 2, [1, 2])
(1, 2, [1])
(1, 2, [1, 99])
(1, 2, [1, 99, 0, 2])
(1, 2, [0, 1, 2, 99])


## 元组诡异赋值的启示
- 单条语句未必是**原子化**操作；
- ```try...exception```捕捉错误后的处理要小心；
- 规范书写程序，简朴的程序是最好的程序；